# Entrega 2 - Implementación y análisis algoritmos ID3 y Random Forest

### Grupo 21:
     - Lucía Bouza  C.I 42897970



## 1. Objetivo

El objetivo de esta tarea es construir clasificadores basados en árboles de decisión y comparar la performance con las implementaciones de Scikit-learn sobre el conjunto de datos QSAR. 

la tarea se divide en 3 partes. La primera implementar el algoritmo ID3. Luego implementar Random Forest utilizando la implementación ID3 de la primera parte. Para finalizar, se propone comparar lo desarrollado con las implementaciones de Scikit-learn, jugando con los hiperparámetros de los métodos. 

Se medirá la performance de los diferentes métodos utilizando las métricas de accurancy, precisión, recall y F1.

Para todas las implmentaciones se utilizará un conjunto de entrenamiento, uno de evaluación para ajuste de los parámetros, y luego uno de test, para determinar la performance final de los algoritmos. Se elige realizarlo de esta manera para evitar sobreajuste. 

## 2. Diseño


    
## 2.1 Preprocesamiento de datos

El conjunto de datos QSAR está compuesto por 1024 atributos binarios de moléculas, clasificándolas en altamente tóxicas y no altamente tóxicas (clase de salida). Todos los datos de los atributos son numéricos, binarios, y no hay datos faltantes. 
La clase de salida tiene solamente dos valores (positive/negative)por lo que lo transformaremos en valores numéricos binarios dado que Scikit-learn solamente acepta atributos numéricos. 

Para realizar este cambio seguimos el ejemplo publicado, utilizando el ordinal encoder.


In [1]:
from sklearn import preprocessing, model_selection, tree, metrics, utils
import pandas as pd

#Carga de Datos
dataset = pd.read_csv('qsar_oral_toxicity.csv', sep=';', prefix='c',header=None)

#preprocesamiento
# creamos un codificador "ordinal" y lo ajustamos a la columna 1024 
enc = preprocessing.OrdinalEncoder()
enc.fit(dataset[['c1024']])

#transformamos la columna 1024 y la guardamos en una nueva columna
dataset['output']  = enc.transform(dataset[['c1024']])

#vemos las 10 primeras filas para observar resultado
dataset.head(10)

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,c1016,c1017,c1018,c1019,c1020,c1021,c1022,c1023,c1024,output
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,negative,0.0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,negative,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,negative,0.0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,negative,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,negative,0.0
5,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,negative,0.0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,negative,0.0
7,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,positive,1.0
8,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,negative,0.0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,negative,0.0


Luego creamos los conjuntos de entrenamiento, validación y test. Se determina que el conjunto de test sea un 20% del Dataset inicial, el conjunto de validación un 16% (20% del conjunto de trainAux) y el conjunto de entrenamiento un 64% del Dataset de inicio. Dado que las clases de salida se encuentran muy desbalanceadas, se determina utilizar estratificación para asegurarnos que tendremos la misma proporción de ambas clases en todos los conjuntos. También se determina no utilizar una semilla (variable random_state), para que los datasets sean diferentes para cada ejecución. 

Para realizar los diferentes conjuntos, ejecutamos el siguiente código.

In [2]:
#creamos conjuntos entrenamiento, validación y test (entrenamiento 64%, validación 16%, test 20%)

#trainAux, test = model_selection.train_test_split(dataset, test_size=0.2)
trainAux, test = model_selection.train_test_split(dataset, stratify=dataset['output'], test_size=0.2)

#creamos conjuntos entrenamiento y validacion (entrenamiento 80%, validacion 20%) a partir del 80% de train
train, validation = model_selection.train_test_split(trainAux, stratify=trainAux['output'], test_size=0.2)

    
## 2.2 Algoritmos

Consideraciones para ambos algoritmos:

Si bien sabemos que nuestro dataset está compuesto de valores binarios, se implementaron los algoritmos para que los atributos de entrada puedan tomar cualquier valor numérico. En un principio se habían hecho ajustes para mejorar los tiempos de ejecución asumiendo que los atributos tomaban solamente valores 0 y 1. Luego se decidió sacrificar estas mejoras en pos de tener un algoritmo más genérico y funcional en diferentes datasets. 

Un cambio importante que debió realizarse fue generar una estructura con los posibles valores de todos los atributos previo a la creación de los árboles. Esto no puede realizarse a la interna del algoritmo de creación del árbol ya que el dataset con el que se trabaja se va particionando. En un momento dado puede darse que en mi dataset de ejemplos falte algún valor del atributo que estoy evaluando, pero igualmente para ese valor es necesario hacer una hoja en el árbol. Si solamente considero los valores que tiene mi atributo en el dataset que estoy trabajando en la recursión, puedo estar perdiéndome estos casos y no estar generando el arbol completo. 


Algoritmo ID3:

Se implementa el algoritmo ID3 descrito en la página 53 del libro de Tom Mitchell e implementándose podas. Se agrega  la posibilidad de poder elegir la máxima cantidad de atributos a evaluar en cada nodo. Si queremos ejecutar el algoritmo ID3 tradicional, en esa variable ponemos la cantidad total de atributos. Sino, podemos colocar el valor que deseemos. Esto nos permite utilizar esta implementación para Random Forest.

Las podas implementadas son dos. La primera poda el árbol y coloca una hoja con el valor más común de la salida cuando no tengo más de 10 datos de ejemplo. La segunda se aplica luego de calcular la ganancia para los atributos (IG). si todos los atributos me dan ganancia cero, entonces hago una hoja con el valor de salida más común para los datos. 

La estructura utilizada para representar un árbol está compuesta de un valor entero (Raiz) y una lista de árboles que respresentan las ramas de dicha raiz. 

A continuación se presenta el pseudocódigo del algoritmo:

```python
    def ID3(Datos,targetAttribute, Atributos, ValsAtributos, maxAtributos):
    Arb = Arbol()
    
    #si todos los valores son iguales, devuelvo ese valor
    Arb.valor = unico Valor
    
    #si no hay atributos, retorno el valor mas comun del atributo objetivo para los datos o  
    # si ejemplos en Datos es menor que 10 (Poda implementada)
    Arb.valor = valor con más ocurrencias
    
    else:
        #seleccionar randómicamente maxAtributos de la lista de Atributos. si tengo menos Atributos que maxAtributos, elijo todos
        subSetAtributos = selección randómica
         
        #seleccionar el mejor atributo, calculando IG para todos los aributos
        Atributo = Atributo con más IG de subSetAtributos
        
        # si IG máximo es 0, retorno el valor mas comun del atributo objetivo para los datos (Poda implementada)
        Arb.valor = valor con más ocurrencias
        
        #Si no:
            #Coloco en la raiz Atributo   
            Arb.valor = Atributo

            #elimino el atributo de la lista
            Atributos = Atributos - Atributo

            #Por cada valor posible del Atributo
            for i in ValoresAtributo    
                # si no hay datos con ese valor, coloco hoja con el valor mas comun de la salida para los datos. 
                    Hoja = Arbol()
                    Hoja.valor = valor con más ocurrencias
                    Arb.Ramas.insert(i,Hoja)
                #sino, llamo recursivo
                else:
                    Arb.Ramas.insert(i, ID3(Ejemplos, targetAttribute, Atributos, ValsAtributos, maxAtributos))      
    return Arb
```

Random Forest:

La implementación del algoritmo de Random Forest se realiza de acuerdo al siguiente pseudocódigo, donde se puede elegir la cantidad de árboles que se desean hacer, la cantidad de ejemplos a tomar para la generación de cada árbol y la cantidad de atributos a evaluar en cada nodo del árbol. En el algoritmo se llama a la implementación de ID3 anterior. La estructura utilizada para la representación del Random Forest es una lista de árboles. 

```python
    def RandomForest(Datos, targetAttribute, Atributos, CantArboles, CantAtributos, CantElementos):
    ListaArboles = []
       
    for i in range(CantArboles):
        #DatasetArbol = Genero DatasetArbol con CantElementos, tomando de forma uniforme de Datos

        #ArbolDecision = llamo a ID3 con el DatasetArbol, los Atributos y la cantAtributos a evaluar en cada nodo
        
        #inserto ArbolDecision en ListaArboles
        
    return ListaArboles
```

## 2.3 Evaluación

Se utilizan las métricas: accurancy, precisión, recall y F1. 
Se decide utilizar métricas adicionales a accurancy ya que la clasificación de las moléculas se encuentra desbalanceada: 741 valores positivos y 8251 valores negativos. 

Precisión y Recuperación nos ayudan a determinar el comportamiento para cada clase de salida.
La precisión mide qué tan bueno es el clasificador cuando dice que un ejemplo es de una determinada clase, mientras que la recuperación mide qué proporción encuentra de los elementos de una clase existentes. La medida-F es la media armónica entre precisión y recall, e intenta combinar ambas en un sólo número. tomaremos F con $\beta$ = 1. 

Calcularemos precisión, recall y F1 para la clase positiva y la negativa, ya que nos interesa poder ver la perfomance que se tiene en ambas clases.

El cálculo de las métricas se realiza manualmente y no con los métodos de Scikit-learn, ya que en un principio se asumió que no se podían utilizar, y al momento que se dijo que si ya se encontraba implementada la evaluación. 


## 3. Experimentación


## 3.1 Resultados algoritmos implementados

Se presentarán a continuación los resultados de las implementaciones A y B contra el conjunto de validación, con diferentes valores de las variables para evaluar los diferentes resultados. En esta tabla se presentan los resultados utilizando una implementación preliminar de ID3 sin podas.

También se presenta una segunda implementación de Random Forest, que la identificamos como Random Forest híbrido. Esta implementación utiliza el mismo algoritmo Random Forest descrito en el inciso anterior, pero con la salvedad que utiliza la implementación de Scikit-learn para construir los árboles de decisión. Se decide explorar este caso, porque al realizar las pruebas de los algoritmos implementados, notamos que la performance de Random Forest era peor que la de ID3. Se decidió realizar esta prueba para observar empíricamente si al utilizar un algoritmo que cree árboles más performantes, el Random Forest implementado era también más performante. Aquí presentaremos los resultados, las conclusiones se verán en el último inciso del documento.  


<table>
  <tr>
    <th>Algoritmo</th>
    <th># Árboles</th>
    <th># Ejemplos</th> 
    <th># Atributos</th>
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr style="font-weight:bold">
    <td>ID3</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>todos</td>
    <td>0,90</td>
    <td>0,28</td>
    <td>0,10</td>
    <td>0,15</td>
    <td>0,92</td>
    <td>0,97</td>
    <td>0,95</td>
  </tr>
  <tr>
    <td>ID3</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>800</td>
    <td>0,90</td>
    <td>0.05</td>
    <td>0.008</td>
    <td>0.01</td>
    <td>0,92</td>
    <td>0,98</td>
    <td>0,95</td>
  </tr>
  <tr>
    <td>ID3</td>
    <td>N/A</td>
    <td>4000</td>
    <td>900</td>
    <td>0,91</td>
    <td>0,11</td>
    <td>0,02</td>
    <td>0,03</td>
    <td>0,92</td>
    <td>0,99</td>
    <td>0,95</td>
  </tr>   
  <tr style="font-weight:bold">
    <td>RF</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>0.91</td>
    <td>0.003</td>
    <td>0.03</td>
    <td>0.01</td>
    <td>0.92</td>
    <td>0.50</td>
    <td>0.64</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>100</td>
    <td>5000</td>
    <td>200</td>
    <td>0.90</td>
    <td>0.003</td>
    <td>0.04</td>
    <td>0.007</td>
    <td>0.91</td>
    <td>0.50</td>
    <td>0.64</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>200</td>
    <td>2000</td>
    <td>100</td>
    <td>0.91</td>
    <td>0.02</td>
    <td>0.004</td>
    <td>0.004</td>
    <td>0.90</td>
    <td>0.47</td>
    <td>0.62</td>
  </tr> 
  <tr>
    <td>RF</td>
    <td>100</td>
    <td>4000</td>
    <td>800</td>
    <td>0.92</td>
    <td>0.001</td>
    <td>0.02</td>
    <td>0.003</td>
    <td>0.92</td>
    <td>0.48</td>
    <td>0.63</td>
  </tr> 
  <tr>
    <td>RF</td>
    <td>100</td>
    <td>4000</td>
    <td>900</td>
    <td>0.92</td>
    <td>0.002</td>
    <td>0.02</td>
    <td>0.004</td>
    <td>0.92</td>
    <td>0.50</td>
    <td>0.65</td>
  </tr>
    <tr>
    <td>RFHibrido</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>0.90</td>
    <td>0.74</td>
    <td>0.39</td>
    <td>0.51</td>
    <td>0.95</td>
    <td>0.98</td>
    <td>0.96</td>
  </tr>
    <caption>Tabla 1 - Comparación algoritmos ID3 y Random Forest implementados</caption>
</table>

Observamos que el mejor desempeño para las pruebas realizadas se da con ID3, considerando todos los ejemplos de entrenamiento y todos los atributos. Para Random Forest, los mejores resultados se obtienen con 100 árboles, 400 atributos evaluados por árbol y 4000 ejemplos tomados por árbol. 

Dado que los algoritmos implementados tienen muy mala performance en la clase positiva, se decide realizar modificaciones al algoritmo ID3, agregando las dos podas explicadas en el inciso 2.2. Esto mejoró notablemente los tiempos de ejecución y los resultados. A continuación presentamos los resultados de las pruebas realizadas tras el cambio en el algoritmo:

<table>
  <tr>
    <th>Algoritmo</th>
    <th># Árboles</th>
    <th># Ejemplos</th> 
    <th># Atributos</th>
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr style="font-weight:bold">
    <td>ID3</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>todos</td>
    <td>0,90</td>
    <td>0,41</td>
    <td>0,41</td>
    <td>0,41</td>
    <td>0,95</td>
    <td>0,95</td>
    <td>0,95</td>
  </tr>
  <tr>
    <td>ID3</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>800</td>
    <td>0,90</td>
    <td>0.41</td>
    <td>0.32</td>
    <td>0.36</td>
    <td>0,94</td>
    <td>0,95</td>
    <td>0,95</td>
  </tr>
  <tr>
    <td>ID3</td>
    <td>N/A</td>
    <td>4000</td>
    <td>900</td>
    <td>0,90</td>
    <td>0,39</td>
    <td>0,39</td>
    <td>0,39</td>
    <td>0,95</td>
    <td>0,95</td>
    <td>0,95</td>
  </tr>   
  <tr style="font-weight:bold">
    <td>RF</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>0.91</td>
    <td>0.46</td>
    <td>0.41</td>
    <td>0.44</td>
    <td>0.94</td>
    <td>0.95</td>
    <td>0.94</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>200</td>
    <td>5000</td>
    <td>100</td>
    <td>0.89</td>
    <td>0.41</td>
    <td>0.35</td>
    <td>0.38</td>
    <td>0.93</td>
    <td>0.95</td>
    <td>0.94</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>200</td>
    <td>2000</td>
    <td>100</td>
    <td>0.89</td>
    <td>0.39</td>
    <td>0.29</td>
    <td>0.33</td>
    <td>0.90</td>
    <td>0.95</td>
    <td>0.93</td>
  </tr> 
  <tr>
    <td>RF</td>
    <td>100</td>
    <td>4000</td>
    <td>800</td>
    <td>0.90</td>
    <td>0.43</td>
    <td>0.35</td>
    <td>0.38</td>
    <td>0.93</td>
    <td>0.95</td>
    <td>0.94</td>
  </tr> 
  <tr>
    <td>RF</td>
    <td>100</td>
    <td>4000</td>
    <td>900</td>
    <td>0.90</td>
    <td>0.38</td>
    <td>0.38</td>
    <td>0.38</td>
    <td>0.94</td>
    <td>0.94</td>
    <td>0.94</td>
  </tr>
    <caption>Tabla 2 - Comparación algoritmos ID3 con Podas y Random Forest</caption>
</table>

Luego de haber detectado los hiperparámetros que mejor funcionan para cada algoritmo, se ejecuta contra el conjunto de Test, obteniéndose los siguientes resultados:

<table>
  <tr>
    <th>Algoritmo</th>
    <th># Árboles</th>
    <th># Ejemplos</th> 
    <th># Atributos</th>
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
 <tr>
    <td>ID3</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>todos</td>
    <td>0,92</td>
    <td>0,52</td>
    <td>0,47</td>
    <td>0,50</td>
    <td>0,95</td>
    <td>0,96</td>
    <td>0,96</td>
  </tr>
    <tr>
    <td>RF</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>0.90</td>
    <td>0.46</td>
    <td>0.40</td>
    <td>0.43</td>
    <td>0.94</td>
    <td>0.95</td>
    <td>0.94</td>
  </tr>
    <caption>Tabla 3 - Resultados ID3 con Podas y Random Forest contra conjunto de test</caption>
</table>

Los resultados son similares a los obtenidos contra el conjunto de evaluación.



## 3.2 Experimentación con Scikit Learn

Se presentarán a continuación los resultados de las implementaciones de Scikit-learn para ID3 y Random Forest, variando algunos de los hiperparámetros. Cabe destacar que muchas de las variables que se pueden modificar no están implementadas en nuestro algoritmo. Realizamos estas pruebas adicionales para experimentar con la librería y comparar entre las diferentes variantes de Scikit-learn, más que para compararlo con nuestra implementación.   


<table>
  <tr>
    <th>Algoritmo</th>
    <th>Criterio</th>
    <th># Árboles</th>
    <th># Ejemplos por árbol</th> 
    <th># Atributos</th>
    <th>Balance de Clases</th>
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
  <tr style="font-weight:bold">
    <td>ID3</td>
    <td>Entropía</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>todos</td>
    <td>No</td>
    <td>0.91</td>
    <td>0.47</td>
    <td>0.50</td>
    <td>0.49</td>
    <td>0.96</td>
    <td>0.95</td>
    <td>0.95</td>
  </tr> 
  <tr>
    <td>ID3</td>
    <td>Entropía</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>800</td>
    <td>No</td>
    <td>0.91</td>
    <td>0.47</td>
    <td>0.50</td>
    <td>0.48</td>
    <td>0.95</td>
    <td>0.95</td>
    <td>0.95</td>
  </tr>
  <tr>
    <td>ID3</td>
    <td>Entropía</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>todos</td>
    <td>balanced</td>
    <td>0.90</td>
    <td>0.46</td>
    <td>0.46</td>
    <td>0.46</td>
    <td>0.95</td>
    <td>0.94</td>
    <td>0.95</td>
  </tr> 
  <tr>
    <td>ID3</td>
    <td>Gini</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>todos</td>
    <td>No</td>
    <td>0.91</td>
    <td>0.50</td>
    <td>0.55</td>
    <td>0.53</td>
    <td>0.96</td>
    <td>0.95</td>
    <td>0.96</td>
  </tr> 
  <tr>
    <td>RF</td>
    <td>Entropía</td>
    <td>100</td>
    <td>1000</td>
    <td>400</td>
    <td>No</td>
    <td>0.92</td>
    <td>0.67</td>
    <td>0.24</td>
    <td>0.35</td>
    <td>0.93</td>
    <td>0.99</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>Entropía</td>
    <td>200</td>
    <td>1000</td>
    <td>400</td>
    <td>No</td>
    <td>0.94</td>
    <td>0.74</td>
    <td>0.24</td>
    <td>0.36</td>
    <td>0.94</td>
    <td>0.99</td>
    <td>0.97</td>
  </tr>
  <tr style="font-weight:bold">
    <td>RF</td>
    <td>Entropía</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>No</td>
    <td>0.94</td>
    <td>0.76</td>
    <td>0.42</td>
    <td>0.54</td>
    <td>0.95</td>
    <td>0.99</td>
    <td>0.97</td>
  </tr>
  <tr style="font-weight:bold">
    <td>RF</td>
    <td>Entropía</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>Balanced</td>
    <td>0.94</td>
    <td>0.73</td>
    <td>0.46</td>
    <td>0.56</td>
    <td>0.95</td>
    <td>0.98</td>
    <td>0.97</td>
  </tr>
  <tr style="font-weight:bold">
    <td>RF</td>
    <td>Entropía</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>Balanced subsample</td>
    <td>0.94</td>
    <td>0.71</td>
    <td>0.43</td>
    <td>0.54</td>
    <td>0.95</td>
    <td>0.98</td>
    <td>0.97</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>Entropía</td>
    <td>100</td>
    <td>1000</td>
    <td>800</td>
    <td>No</td>
    <td>0.93</td>
    <td>0.74</td>
    <td>0.30</td>
    <td>0.43</td>
    <td>0.94</td>
    <td>0.99</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>Entropía</td>
    <td>100</td>
    <td>1000</td>
    <td>200</td>
    <td>No</td>
    <td>0.93</td>
    <td>0.79</td>
    <td>0.25</td>
    <td>0.38</td>
    <td>0.93</td>
    <td>0.99</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>Entropía</td>
    <td>200</td>
    <td>4000</td>
    <td>800</td>
    <td>No</td>
    <td>0.93</td>
    <td>0.71</td>
    <td>0.33</td>
    <td>0.45</td>
    <td>0.94</td>
    <td>0.99</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>Gini</td>
    <td>100</td>
    <td>1000</td>
    <td>400</td>
    <td>No</td>
    <td>0.93</td>
    <td>0.67</td>
    <td>0.18</td>
    <td>0.28</td>
    <td>0.93</td>
    <td>0.99</td>
    <td>0.96</td>
  </tr>
  <tr>
    <td>RF</td>
    <td>Gini</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>No</td>
    <td>0.93</td>
    <td>0.79</td>
    <td>0.41</td>
    <td>0.54</td>
    <td>0.94</td>
    <td>0.99</td>
    <td>0.96</td>
  </tr>
    <caption>Tabla 4 - Comparación algoritmos ID3 y Random Forest de Scikit-learn</caption>
</table>

Observando la tabla vemos que para todos los casos el algoritmo tiene peor performance para detectar los casos positivos. Esto se da por la naturaleza del dataset, donde una gran proporción de los ejemplos son de clase negativa en la salida. Aún cuando utilizamos la funcionalidad del balance de clases, los resultados no mejoran significativamente. 

Observamos que el mejor desempeño para las pruebas realizadas se da con Random Forest, con 100 árboles, 400 atributos evaluados por árbol y 4000 ejemplos tomados por árbol, utilizando la métrica de entropía para la selección de atributos. 

Los tiempos de ejecución de estos métodos son del orden de los milisegundos, mientras que las implementaciones realizadas son del orden de minutos para ID3, y horas para Random Forest. 



## 3.3 Comparación resultados

Se presentarán a continuación los mejores resultados de las implementaciones A y B, junto con los mejores resultados de Scikit learn. Para todos los casos el criterio de elección de atributos para la construcción del árbol es entropía.  


<table>
   <tr>
    <th>Algoritmo</th>
    <th># Árboles</th>
    <th># Ejemplos por árbol</th> 
    <th># Atributos</th>
    <th>Accurancy</th>
    <th>Precisión 1</th>
    <th>Recall 1</th>
    <th>F1 1</th>
    <th>Precisión 0</th>
    <th>Recall 0</th>
    <th>F1 0</th>
  </tr>
 <tr>
    <td>ID3 c/podas</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>todos</td>
    <td>0,90</td>
    <td>0,41</td>
    <td>0,41</td>
    <td>0,41</td>
    <td>0,95</td>
    <td>0,95</td>
    <td>0,95</td>
  </tr>  
  <tr>
    <td>RF c/ID3 podas</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>0.91</td>
    <td>0.46</td>
    <td>0.41</td>
    <td>0.44</td>
    <td>0.94</td>
    <td>0.95</td>
    <td>0.94</td>
  </tr>
  <tr>
    <td>RF Balanced Scikit</td>
    <td>100</td>
    <td>4000</td>
    <td>400</td>
    <td>0.94</td>
    <td>0.73</td>
    <td>0.46</td>
    <td>0.56</td>
    <td>0.95</td>
    <td>0.98</td>
    <td>0.97</td>
  </tr>
    <tr>
    <td>ID3 Scikit</td>
    <td>N/A</td>
    <td>|train|</td>
    <td>todos</td>
    <td>0.91</td>
    <td>0.47</td>
    <td>0.50</td>
    <td>0.49</td>
    <td>0.96</td>
    <td>0.95</td>
    <td>0.95</td>
  </tr> 
    <caption>Tabla 5 - Comparación algoritmos implementados contra Scikit Learn</caption>
</table>

Podemos observar que si bien la performance de los algoritmos Scikit-learn es mejor, se da que tanto para los algoritmos implementados como para los de Scikit-learn, los hiperparámetros que mejor funcionan son los mismos. 

En el caso de ID3, se da lo que se esperaba, y es un mejor rendimiento cuando se utiliza todo el conjunto de entrenamiento y se evalúan todos los atributos.

En el caso de Random Forest los hiperparámetros que dan mejores resultados es la construcción de 100 árboles, donde se evalúan en cada uno 400 atributos y se utilizan 4000 ejemplos de entrenamiento elegidos de forma randómica, tanto atributos como ejemplos. 

La performance de la implementación de Scikit es mejor, sobre todo en la clase de los positivos para Random Forest. Para ID3 la implementación de nuestro algoritmo se acerca bastante a la performance del de Scikit-learn.


## 3.4 Ejecución de ID3

Aquí se da la posibilidad de ejecutar la implementación del algoritmo ID3. Se permite elegir la cantidad de máxima de atributos a evaluar en cada nodo y la cantidad de ejemplos a utilizar para poder correrlo en menor tiempo. Si se desea evaluar contra el conjunto de validación, cambiar test por validation. 

In [3]:
import ID3

# Aqui se pueden modificar los parámetros para ejecutar ID3
CantidadAtributos = 1024
CantidadEjemplos = train.shape[0]

ID3.CreacionYEvaluacionID3(dataset, train, test, CantidadEjemplos, CantidadAtributos)


Accurancy: 0.9216231239577543
ejemplos positivos: 148
ejemplos positivos acertados: 70
Precision 1: 0.5263157894736842
Recall 1: 0.47297297297297297
F1 1: 0.498220640569395
ejemplos Negativos: 1651
ejemplos Negativos acertados: 1588
Precision 0: 0.9531812725090036
Recall 0: 0.9618413082980012
F1 0: 0.9574917093759421


## 3.5 Ejecución de Random Forest

Aquí se da la posibilidad de ejecutar la implementación del algoritmo Random Forest. Se permite elegir la cantidad máxima de atributos a evaluar en cada nodo, la cantidad de ejemplos a utilizar para la generación de cada árbol y la cantidad de árboles a generar. Si se desea evaluar contra el conjunto de validación, cambiar test por validation. 

In [4]:
import RandForest

# Aqui se pueden modificar los parámetros para ejecutar Random Forest
CantArboles= 100
CantAtributos= 400
CantElementos = 4000

RandForest.CreacionYEvaluacionRandomForest(dataset, train, test, CantArboles, CantAtributos, CantElementos)

Accurancy: 0.9043913285158421
ejemplos positivos: 148
ejemplos positivos acertados: 74
Precision 1: 0.4625
Recall 1: 0.4088397790055249
F1 1: 0.43401759530791795
ejemplos Negativos: 1651
ejemplos Negativos acertados: 1638
Precision 0: 0.9386819484240687
Recall 0: 0.9501160092807425
F1 0: 0.9443643701354857


## 4. Conclusión

Con la implementación de este laboratorio, se pueden comprobar de forma empírica algunos de los conceptos estudiados en el teórico.

Con respecto a las métricas:
Si bien en primera instancia al obtener la métrica de accurancy se puede presuponer que el algoritmo funciona realmente bien, es importante conocer la naturaleza de nuestro dataset ya que podríamos estar engañándonos. 
Para el caso del dataset QSAR, los resultados de salida se encuetran muy desbalanceados. Cuando generamos los clasificadores, sea Random Forest o ID3, encontramos que tiene una altísima precisión para los casos más abundantes (cuando la molécula no es altamente tóxica), pero muy baja para los casos positivos (cuando la molécula es altamente tóxica). 
Esto quiere decir que la gran mayoría de las veces clasifico a las moléculas como no tóxicas, y estoy en lo cierto la gran mayoría de las veces porque los ejemplos son casi todos de este tipo; pero el algoritmo tiene muy poca capacidad para detectar las moléculas altamente tóxicas, que seguramente sea el dato más interesante. 

Es por eso que importa tener las métricas de Precisión, Recall y F1 para las diferentes clases, ya que nos da un mejor entendimiento de la performance de los algoritmos.

Con respecto a los algoritmos:
Notamos que la performance en detectar las clases positivas siempre es muchísimo más baja que para detectar las negativas, independientemente del algoritmo y los parámetros utilizados. 
También notamos que la performance de los algoritmos de SciKit learn es mejor que la de los algoritmos implementados. Con esto nos referimos a la performance en la clasificación de objetos, pero también cabe destacar que los tiempos de respuesta son de ordenes mucho más bajos que los desarrollados.

Comparando los diferentes algoritmos para las implementaciones de Scikit learn, vemos que Random Forest tiene mejor performance que ID3. Encontramos que mejora al tomar mayor cantidad de ejemplos para entrenar cada árbol, y que la cantidad de atributos tomados para construir cada árbol, no impacta tan significativamente.

En el caso de nuestra implementación, el algoritmo ID3 sin podas se comporta mejor que Random Forest. Suponíamos que el mal comportamiento de Random Forest se debía a que ID3 tiene baja performance en la clase positiva, dada la naturaleza desbalanceada del dataset. Al utilizarlo repetidas veces para la creación de los árboles en RF, y luego aplicar votación entre los árboles para clasificar, el error se ve intensificado y afecta en mayor proporción a Random Forest. Fue por esta razón que se optó realizar el experimento del Random Forest híbrido, obteniendo resultados similares al Random Forest de Scikit-learn, por lo que la suposición parecería ser correcta. 

Al tener nuestros algoritmos muy mala performance en la clase positiva, y al haber detectado que el problema radicaba en ID3, se decidió implementar podas para evitar sobreajuste. Al realizar las pruebas se vió que esto mejoraba notablemente la performance tanto de ID3 como de Random Forest, ID3 alcanzando valores similares a la implementación de Scikit-learn, y Random Forest acercándose muchísimo en comparación a su antigua implementación. 




